In [1]:
# Normal decorator example without arguments
from functools import wraps,partial
def deco(function):
    
    @wraps(function)
    def wrapper(*args, **kwargs):
        print("extra")
        function(*args, **kwargs)
    return wrapper

# can add with option @deco
def add(a,b):
    print("result: ",a + b)

add = deco(add)
print(add(2,4))

extra
result:  6
None


In [2]:
add(3,4)

extra
result:  7


In [3]:
add

<function __main__.add(a, b)>

In [4]:
# decorators with parameter
# problem with it is when no parameter is passed u have to give a callable sign ()
# calling without @ will be add = deco_param(param1="",param2="")(add)

def deco_param(param1='',param2=''):
    
    def deco(function):
        
        @wraps(function)
        def wrapper(*args, **kwargs):
            msg = "this is extra info "
            if param1:
                msg = msg+param1+" "
            if param2:
                msg = msg+param2
            print(msg)
            return function(*args,**kwargs)
        return wrapper
    return deco

@deco_param() # have to give () sign otherwise throughs a error
def add1(x,y):
    return x+y

@deco_param(param1="number1")
def add2(x,y):
    return x+y
            
    
@deco_param(param1="number1", param2="number2")
def add3(x,y):
    return x+y

def add4(x,y):
    return x+y

@deco_param # without a ()
def add5(x,y):
    return x+y

add4 = deco_param(param1="info1",param2="info2")(add4)
            

In [5]:
add1(3,4)

this is extra info 


7

In [6]:
add2(3,4)

this is extra info number1 


7

In [7]:
add3(3,4)

this is extra info number1 number2


7

In [8]:
add4(3,4)

this is extra info info1 info2


7

In [9]:
# calling like add5 = deco_param(add5)(3,4)
# deco function takes 1 argument which is a function
# the proper formet should be final_function = deco_param(parameters)(function_name)
# thats why the error is produced
add5(3,4)

TypeError: deco() takes 1 positional argument but 2 were given

In [10]:
# so for handling () problem i have modify some code like below

def deco_param_version2(func=None, param1='',param2=''):
    
    def deco(function):
        
        @wraps(function)
        def wrapper(*args, **kwargs):
            msg = "this is extra info "
            if param1:
                msg = msg+param1+" "
            if param2:
                msg = msg+param2
            print(msg)
            return function(*args,**kwargs)
        return wrapper
    
    # see the change here
    if func:
        return deco(func)
    
    
    return deco

@deco_param_version2() # with ()
def add1_v2(x,y):
    return x+y

@deco_param_version2(param1="number1")
def add2_v2(x,y):
    return x+y
            
    
@deco_param_version2(param1="number1", param2="number2")
def add3_v2(x,y):
    return x+y

def add4_v2(x,y):
    return x+y

@deco_param_version2 # without ()
def add5_v2(x,y):
    return x+y

add4_v2 = deco_param(param1="info1",param2="info2")(add4_v2)

In [11]:
add1_v2(2,3)

this is extra info 


5

In [12]:
add2_v2(2,3)

this is extra info number1 


5

In [13]:
add3_v2(2,3)

this is extra info number1 number2


5

In [14]:
add4_v2(2,3)

this is extra info info1 info2


5

In [15]:
add5_v2(2,3)

this is extra info 


5

In [16]:
# there is another elegant way of doing this proposed by David Beazly using partial function


    
def deco(function=None, *, param1='',param2=''):
    
    # see the change here
    if function is None:
        return partial(deco, param1=param1, param2=param2)

    @wraps(function)
    def wrapper(*args, **kwargs):
        msg = "this is extra info "
        if param1:
            msg = msg+param1+" "
        if param2:
            msg = msg+param2
        print(msg)
        return function(*args,**kwargs)
    return wrapper
    
    

@deco() # with ()
def add1_v3(x,y):
    return x+y

@deco(param1="number1")
def add2_v3(x,y):
    return x+y
            
    
@deco(param1="number1", param2="number2")
def add3_v3(x,y):
    return x+y

def add4_v3(x,y):
    return x+y

@deco # without ()
def add5_v3(x,y):
    return x+y

add4_v3 = deco(add4_v3, param1="info1",param2="info2")

In [17]:
add1_v3(2,3)

this is extra info 


5

In [18]:
add2_v3(2,3)

this is extra info number1 


5

In [19]:
add3_v3(2,3)

this is extra info number1 number2


5

In [20]:
add4_v3(2,3)

this is extra info info1 info2


5

In [21]:
add5_v3(2,3)

this is extra info 


5

In [25]:
# class decorators example

def debug(func=None, * , prefix=""):
    if func is None:
        return partial(debug,prefix=prefix)
    
    msg = prefix + func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper
        
        

def debug_class_methods(cls=None, *, prefix=""):
    if cls is None:
        return partial(debug_class_methods, prefix=prefix)
    for key,value in vars(cls).items():
        if callable(value):
            setattr(cls,key,debug(value,prefix=prefix))
    return cls

In [30]:
@debug_class_methods
class Spam:
    def foo(self):
        pass
    def bar(self):
        pass
    
    
@debug_class_methods(prefix="@@@@@@")
class Spam2:
    def a(self):
        pass
    def b(self):
        pass

In [31]:
s = Spam()
s.foo()

Spam.foo


In [32]:
s = Spam2()
s.a()

@@@@@@Spam2.a


In [33]:
def debug_attribute(cls):
    orig_attribute = cls.__getattribute__
    def __getattribute__(self,name):
        print(f"Get : {name}")
        return orig_attribute(self,name)
    cls.__getattribute__ = __getattribute__
    return cls

@debug_attribute
class DebugExample:
    def __init__(self):
        self.a = 34
        self.b = 55
    def foo(self):
        pass


In [34]:
s = DebugExample()

In [35]:
s.a

Get : a


34

In [36]:
s.b

Get : b


55

In [37]:
s.foo()

Get : foo
